# 第5章 利用上下文信息

## 时间上下文信息

### 时间效应简介

时间信息对用户兴趣的影响表现在以下几个方面：
- **用户兴趣是变化的：**年龄的增长、工作年限的增加，如果要准确预测用户现在的兴趣，就应该关注用户最近的行为，因为用户最近的行为最能体现他现在的兴趣。
- **物品也是有生命周期的：**电影（物品）上映的时间，也可能受新闻事件的影响，当决定在某个时刻给某个用户推荐某个物品时，需要考虑该物品在该时刻是否已经过时了。
- **季节效应：**季节效应主要反应了时间本身对用户兴趣的影响，节日也是一种季节效应。

### 系统时间特性的分析
包含时间信息的用户行为数据集由一系列三元组构成，其中每个三元组$(u,i,t)$代表了用户$u$在时刻$t$对物品$i$产生过行为。
通过统计如下信息研究系统的时间特性：
- 数据集每天独立用户数的增长情况
- 系统的物品变化情况
- 用户访问情况

In [1]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

PROJECT_ROOT = os.path.dirname(sys.path[0])

In [2]:
user_bookmark_path = os.path.join(PROJECT_ROOT, "src/data/delicious-2k/user_taggedbookmarks-timestamps.dat")
bookmarks_path = os.path.join(PROJECT_ROOT, "src/data/delicious-2k/bookmarks.dat")
user_bookmark_dataset = pd.read_table(user_bookmark_path, sep='\t', engine='python')
bookmarks_dataset = pd.read_table(bookmarks_path, sep='\t', engine='python')

bookmarks_dataset.rename(columns={'id':'bookmarkID'},inplace=True)
dataset = pd.merge(user_bookmark_dataset, bookmarks_dataset, how='left', on=['bookmarkID'])
dataset = dataset[['userID', 'bookmarkID', 'tagID', 'urlPrincipal', 'timestamp']]

dataset.head()

,userID,bookmarkID,tagID,urlPrincipal,timestamp
0,8,1,1,www.ifla.org,1289255362000
1,8,2,1,archive.ifla.org,1289255159000
2,8,7,1,www.edselect.com,1289238901000
3,8,7,6,www.edselect.com,1289238901000
4,8,7,7,www.edselect.com,1289238901000


In [4]:
def delete_tag_id(dataset, site_name):
    return dataset[dataset['urlPrincipal'].str.find(site_name) != -1].drop(labels=['urlPrincipal', 'tagID'],axis=1).drop_duplicates()

nytimes_dataset = delete_tag_id(dataset, 'nytimes.com')
youtube_dataset = delete_tag_id(dataset, 'youtube.com')
wikipedia_dataset = delete_tag_id(dataset, 'wikipedia.com')
sourceforge_dataset = delete_tag_id(dataset, 'sourceforge.com')
blogspot_dataset = delete_tag_id(dataset, 'blogspot.com')

dataset_describe = []
dataset_describe.append((nytimes_dataset['userID'].nunique(), nytimes_dataset['bookmarkID'].nunique()))
dataset_describe.append((youtube_dataset['userID'].nunique(), youtube_dataset['bookmarkID'].nunique()))
dataset_describe.append((wikipedia_dataset['userID'].nunique(), wikipedia_dataset['bookmarkID'].nunique()))
dataset_describe.append((sourceforge_dataset['userID'].nunique(), sourceforge_dataset['bookmarkID'].nunique()))
dataset_describe.append((blogspot_dataset['userID'].nunique(), blogspot_dataset['bookmarkID'].nunique()))

pd.DataFrame(
        data = dataset_describe,
        index = ['nytimes', 'youtube', 'wikipedia', 'sourceforge', 'blogspot'],
        columns = ["user_total","item_total"]
)

,user_total,item_total
nytimes,546,877
youtube,711,1072
wikipedia,0,0
sourceforge,0,0
blogspot,989,1459


### 推荐系统的实时性

1. 实时推荐系统不能每天都给所有用户离线计算推荐结果，然后在线展示昨天计算出来的结果，所以，要求在每个用户访问推荐系统时，都根据用户这个时间点钱的行为**实时计算**推荐列表。
2. 推荐算法需要平衡考虑用户的近期行为和长期行为，既要让推荐列表反应出用户经期行为所体现的兴趣，又不能让推荐列表完全受用户近期行为的影响，要保证推荐列表对用户兴趣预测的**延续性**。

### 推荐算法的时间多样性

时间多样性：推荐系统每天推荐结果的变化程度。时间多样性高的推荐系统中，用户会经常看到不同的推荐结果。

**问题：**如何在不损失精度的情况下，提高推荐结果的时间多样性。  
提高推荐结果时间多样性需要分两步：  
1. 需要保证推荐系统能够在用户有了新的行为后，及时调整推荐结果。
2. 需要保证推荐系统在用户没有新的行为时，也能够经常变化一下结果。

解决如果用户没有行为，如何保证给用户的推荐结果具有一定的时间多样性：  
1. 在生成推荐结果时，加入一定的随机性。
2. 记录用户每天看到的推荐结果，然后在每天给用户进行推荐时，对他前几天看到过很多次的推荐结果进行适当地降权。
3. 每天给用户使用不同的推荐算法。

### 时间上下文推荐算法

#### 最近最热门

给定时间$T$、物品$i$最近的流行度$n_i(T)$定义为：$$n_i(T)=\sum_{(u,i,t) \in \text{Train}, t < T} \frac{1}{1 + \alpha(T-t)}$$其中，$\alpha$是时间衰减参数。

In [5]:
from main.util import delicious_reader

train_dataset, test_dataset = delicious_reader.load_data(bookmarks_path, user_bookmark_path, "nytimes.com")

In [ ]:
from main.chapter5.recent_popularity import RecentPopular

model = RecentPopular(train_dataset, )